In [ ]:
# Import the necessary libraries
from google.colab import auth
from google.colab import output
import time

# Authenticate user credentials
auth.authenticate_user()

# Set a timer for 6 hours
timer = 6 * 60 * 60  # in seconds

# Start time
start = time.time()

# Loop until the timer is up
while time.time() - start < timer:
  # Check if the runtime is still connected
  if not output.is_streaming():
    # If not, reconnect to the runtime
    print("Reconnecting to runtime...")
    output.serve_kernel_port_as_iframe()
  # Wait for 10 minutes before checking the connection status again
  time.sleep(600)
  
# When the timer is up, print a message and disconnect the runtime
print("Code execution completed. Disconnecting from runtime...")
output.eval_js('document.querySelector("#connect-button").click()')


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import zipfile
# zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Yes No/yes-no.zip", 'r')
# zip_ref.extractall(r"/content/Yes No")
# zip_ref.close()


In [29]:
import numpy as np
import cv2
import random
import os
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from sklearn.model_selection import train_test_split
import pandas as pd

DIRECTORY = r'/content/drive/MyDrive/rsna-pneumonia-detection-challenge/stage_2_train_png_images'
CATEGORIES = ['Aug_Pneumonia', 'Aug_Normal']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2

# define the path to the input and output directories
input_path = "/content/drive/MyDrive/rsna-pneumonia-detection-challenge/stage_2_train_png_images/non-pneumonia"
output_path = "/content/drive/MyDrive/rsna-pneumonia-detection-challenge/stage_2_train_png_images/Aug_Normal"

# create the output directory if it does not exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# define the augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=(30), 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True, 
    fill_mode='nearest')
kj = 0
# loop over the images in the input directory
for filename in os.listdir(input_path):
    # load the image from disk
    image = cv2.imread(os.path.join(input_path, filename))
    # apply the augmentation and save the image to disk
    for idx, batch in enumerate(datagen.flow(image.reshape(1, *image.shape), batch_size=1)):
        # get the augmented image
        augmented_image = batch[0].astype("uint8")
        # save the image to disk
        cv2.imwrite(os.path.join(output_path, f"{filename[:-4]}_{idx}.png"), augmented_image)
        # break out of the loop if we have generated the desired number of images
        kj += 1
        print(kj)
        if idx == 1:
            break


In [27]:
import os

dir_path = "/content/drive/MyDrive/rsna-pneumonia-detection-challenge/stage_2_train_png_images/Aug_Normal"

num_images = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
print(f"Number of images in directory: {num_images}")

Number of images in directory: 12000


In [32]:
from keras.preprocessing.image import ImageDataGenerator

# Define the image size, batch size, and number of classes
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 2

# Define the data generator for training data
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
        DIRECTORY,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=CATEGORIES,
        subset='training')

# Define the data generator for testing data
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_generator = test_datagen.flow_from_directory(
        DIRECTORY,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=CATEGORIES,
        subset='validation')


Found 18935 images belonging to 2 classes.
Found 4733 images belonging to 2 classes.


In [ ]:
# Works with dense net


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.applications.densenet import DenseNet121

# # Load the DenseNet model
# densenet = DenseNet121(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# # Freeze the DenseNet layers
# for layer in densenet.layers:
#     layer.trainable = False

# # Create a Sequential model
# model = Sequential()

# # Add the DenseNet model to the Sequential model
# model.add(densenet)

# # Add a Flatten layer to the Sequential model
# model.add(Flatten())

# # Add a Dense layer to the Sequential model
# model.add(Dense(128, activation='relu'))

# # Add a Dense layer with softmax activation to the Sequential model
# model.add(Dense(2, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.densenet import DenseNet201

# Load the DenseNet model
densenet = DenseNet201(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the first few layers of the DenseNet model
for layer in densenet.layers[:100]:
    layer.trainable = False

# Create a Sequential model
model = Sequential()

# Add the DenseNet model to the Sequential model
model.add(densenet)

# Add a Flatten layer to the Sequential model
model.add(Flatten())

# Add a Dense layer with 512 units and 'relu' activation
model.add(Dense(64, activation='relu'))

# Add a Dropout layer with a rate of 0.5
# model.add(Dropout(0.5))

# Add a Dense layer with 256 units and 'relu' activation
model.add(Dense(256, activation='relu'))
# model.add(Dense(256, activation='relu'))
# Add a Dropout layer with a rate of 0.5
# model.add(Dropout(0.5))

# Add a Dense layer with softmax activation
model.add(Dense(2, activation='softmax'))

# Compile the model with a smaller learning rate and L2 regularization
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [68]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
 flatten_13 (Flatten)        (None, 94080)             0         
                                                                 
 dense_47 (Dense)            (None, 64)                6021184   
                                                                 
 dense_48 (Dense)            (None, 256)               16640     
                                                                 
 dense_49 (Dense)            (None, 2)                 514       
                                                                 
Total params: 24,360,322
Trainable params: 23,326,594
Non-trainable params: 1,033,728
_________________________________________________________________


In [ ]:


# Train the model
history = model.fit(
        train_generator,
        epochs=100,
        validation_data=test_generator)

Epoch 1/100
592/592 [==============================] - 375s 464ms/step - loss: 0.6365 - accuracy: 0.7171 - val_loss: 0.5387 - val_accuracy: 0.7520
Epoch 2/100
592/592 [==============================] - 271s 458ms/step - loss: 0.5027 - accuracy: 0.7572 - val_loss: 0.7138 - val_accuracy: 0.6201
Epoch 3/100
592/592 [==============================] - 285s 481ms/step - loss: 0.4876 - accuracy: 0.7740 - val_loss: 0.5146 - val_accuracy: 0.7758
Epoch 4/100
592/592 [==============================] - 274s 462ms/step - loss: 0.4888 - accuracy: 0.7717 - val_loss: 0.6390 - val_accuracy: 0.6955
Epoch 5/100
592/592 [==============================] - 274s 463ms/step - loss: 0.4810 - accuracy: 0.7767 - val_loss: 0.4853 - val_accuracy: 0.7566
Epoch 6/100
592/592 [==============================] - 272s 459ms/step - loss: 0.4769 - accuracy: 0.7810 - val_loss: 0.4590 - val_accuracy: 0.7925
Epoch 7/100
592/592 [==============================] - 272s 459ms/step - loss: 0.4686 - accuracy: 0.7801 - val_loss: 0

In [ ]:
from keras.models import load_model

with open('history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

# Save the model
model.save('my_model.h5')


# Load the saved model from disk
loaded_model = load_model('my_model.h5')


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:

# Plot training & validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [39]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc* 100, "%")

15/15 [==============================] - 1s 98ms/step - loss: 2.7649 - accuracy: 0.6674
Test accuracy: 66.73684120178223 %


In [ ]:
model.summary()